In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

In [2]:
# 문서 로드와 쪼개기 / 임베딩 생성 및 캐시

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./article.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

In [3]:
# 메모리와 문서를 이용한 프롬프트

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=20,
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

In [4]:
# 체인 연결 / 체인 호출 함수 정의

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    print(result)
    memory.save_context({"input": question}, {"output": result.content})

In [13]:
# 체인에 질문하여 테스트

invoke_chain("Who is Yoon Suk Yeol?")
invoke_chain("What is the main topic of this article?")
invoke_chain("Who is Woo Won-shik?")

content="Yoon Suk Yeol is the President of South Korea mentioned in the article, who is facing political turmoil and challenges related to his administration's reform initiatives."
content='The main topic of the article is the challenges and setbacks facing the Yoon administration in South Korea, particularly regarding stalled reform initiatives in health care and pensions, as well as the political turmoil following the impeachment of President Yoon Suk-yeol.'
content='Woo Won-shik is the National Assembly Speaker mentioned in the article.'


In [15]:
#메모리를 출력하여 메모리가 체인에 적용되었는지 확인
 
load_memory({})

[HumanMessage(content='Is Mr. Suk Yeol guilty?'),
 AIMessage(content="I don't know."),
 HumanMessage(content='What is the main topic of this article?'),
 AIMessage(content='The main topic of the article is the challenges and setbacks facing the Yoon administration in South Korea, particularly regarding stalled reform initiatives in health care and pensions, as well as the political turmoil following the impeachment of President Yoon Suk-yeol.'),
 HumanMessage(content='Who is Woo Won-shik?'),
 AIMessage(content='Woo Won-shik is the National Assembly Speaker mentioned in the article.'),
 HumanMessage(content='How do you feel about What the President did?'),
 AIMessage(content="I don't know."),
 HumanMessage(content='What is the main topic of this article?'),
 AIMessage(content='The main topic of the article is the challenges and setbacks facing the Yoon administration in South Korea, particularly regarding stalled reform initiatives in health care and pensions, as well as the political t